In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
from pathlib import Path

In [3]:

# from fastai2.text.all import *

In [4]:
bs=128
# data_path = Config.data_path()

In [5]:
data_path = Path('/home/quan/.fastai/data')
data_path

PosixPath('/home/quan/.fastai/data')

In [6]:
lang = 'vi'

In [7]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [8]:
name,path,lm_fns

('viwiki',
 PosixPath('/home/quan/.fastai/data/viwiki'),
 ['vi_wt', 'vi_wt_vocab'])

# Get data

## wiki funcs

In [11]:
from fastai.basics import *
import re


def get_wiki(path,lang):
    name = f'{lang}wiki'
    if (path/name).exists():
        print(f"{path/name} already exists; not downloading")
        return

    xml_fn = f"{lang}wiki-latest-pages-articles.xml"
    zip_fn = f"{xml_fn}.bz2"

    if not (path/xml_fn).exists():
        print("downloading...")
        download_url(f'https://dumps.wikimedia.org/{name}/latest/{zip_fn}', path/zip_fn)
        print("unzipping...")
        bunzip(path/zip_fn)

    with working_directory(path):
        if not (path/'wikiextractor').exists():
            print('Do these steps below and come back!')
            return
#             os.system('git clone -n https://github.com/attardi/wikiextractor.git')
#             os.system('cd wikiextractor')
#             os.system('git checkout -b working_branch e4abb4cbd0')
#             os.system('cd ..')
        print("extracting...")
        os.system("python wikiextractor/WikiExtractor.py --processes 4 --no_templates " +
            f"--min_text_length 1800 --filter_disambig_pages --log_file log -b 100G -q {xml_fn}")
    shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
    shutil.rmtree(path/'text')


def split_wiki(path,lang,encoding='utf-8'):
    dest = path/'docs'
    name = f'{lang}wiki'
    if dest.exists():
        print(f"{dest} already exists; not splitting")
        return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'<doc id="\d+" url="https://{lang}.wikipedia.org/wiki\?curid=\d+" title="([^"]+)">')
    lines = (path/name).open(encoding=encoding)
    f=None

    for i,l in enumerate(lines):
        if i%100000 == 0: print(i)
        if l.startswith('<doc id="'):
            title = title_re.findall(l)[0].replace('/','_')
            if len(title)>150: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w', encoding=encoding)
        elif l.startswith('</doc>'): continue
        else: f.write(l)
    f.close()
    return dest

## continue

In [12]:
get_wiki(path,lang)

extracting...


In [13]:
!head -n4 {path}/{name}

<doc id="13" url="https://vi.wikipedia.org/wiki?curid=13" title="Tiếng Việt">
Tiếng Việt

Tiếng Việt (chữ Nôm: ), cũng được gọi là tiếng Việt Nam (chữ Nôm: 㗂越南), tiếng Kinh (chữ Nôm: 㗂京) hay Việt ngữ (chữ Nôm: 越語) là ngôn ngữ của người Việt và là ngôn ngữ chính thức tại Việt Nam. Đây là tiếng mẹ đẻ của khoảng 85% dân cư Việt Nam, cùng với hơn 4 triệu Việt kiều. Tiếng Việt còn là ngôn ngữ thứ hai của các dân tộc thiểu số tại Việt Nam, và được công nhận là một ngôn ngữ dân tộc thiểu số tại Cộng hòa Séc.


In [14]:
dest = split_wiki(path,lang)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000


In [15]:
dest

Path('/home/quan/.fastai/data/viwiki/docs')

In [16]:
dest.ls()[:5]

(#5) [Path('/home/quan/.fastai/data/viwiki/docs/Doug Emhoff.txt'),Path('/home/quan/.fastai/data/viwiki/docs/HMS Myngs (R06).txt'),Path('/home/quan/.fastai/data/viwiki/docs/Xuân La (phường).txt'),Path('/home/quan/.fastai/data/viwiki/docs/Cung điện Goldstein.txt'),Path('/home/quan/.fastai/data/viwiki/docs/Tứ khố toàn thư.txt')]